In [190]:
import requests
import pandas as pd
import schedule
import time
from datetime import datetime, timedelta
from xgboost import XGBRegressor
from datetime import datetime
import pandas as pd
import os
from functools import partial
import pickle
import numpy as np
import time
import threading
import time
import schedule
import telebot
from telebot import types
from aiogram import Bot, Dispatcher

In [110]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [183]:
user_chat_ids = set()

def send_message(chat_id, message):
    bot.send_message(chat_id, message)

bot = telebot.TeleBot('6440517312:AAGRtFDdiKZbNNvUZ2LhWW1hfQwJHdOIMGw', parse_mode=None)

def schedule_func_1(chat_id):
    bot.send_message(chat_id, 'привет хайпарики! чек как у вас настроение?')

def f(chat_id):
    schedule.every(1).minutes.do(schedule_func_1, chat_id=chat_id)
    while True:
        schedule.run_pending()
        time.sleep(1)

@bot.message_handler(commands=['start'])
def start(message: types.Message):
    chat_id = message.chat.id
    bot.send_message(chat_id, 'хай')
    thread = threading.Thread(target=f, args=(chat_id,))
    thread.start()

if __name__ == '__main__':
    bot.polling(none_stop=True)

In [185]:
bot_token = '6440517312:AAGRtFDdiKZbNNvUZ2LhWW1hfQwJHdOIMGw'
bot = telebot.TeleBot(bot_token)

In [147]:
def preprocess_data(df):
    df['DATE'] = df['DATE'].astype(str)
    df['TIME'] = df['TIME'].astype(str)
    df['datetime'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'], format='%d/%m/%y %H:%M', errors='coerce')
    df['datetime'] = pd.to_datetime(df['datetime'])

    df['day'] = df['datetime'].dt.day
    df['day_of_week'] = df['datetime'].dt.dayofweek

    df['Mean_1d'] = df['CLOSE'].rolling(window=24).mean()
    df['Max_1d'] = df['CLOSE'].rolling(window=24).max()
    df['Min_1d'] = df['CLOSE'].rolling(window=24).min()
    df['Std_1d'] = df['CLOSE'].rolling(window=24).std()
    df['Range_1d'] = df['Max_1d'] - df['Min_1d']

    df['Mean_3d'] = df['CLOSE'].rolling(window=72).mean()
    df['Max_3d'] = df['CLOSE'].rolling(window=72).max()
    df['Min_3d'] = df['CLOSE'].rolling(window=72).min()
    df['Std_3d'] = df['CLOSE'].rolling(window=72).std()
    df['Range_3d'] = df['Max_3d'] - df['Min_3d']

    df['Mean_7d'] = df['CLOSE'].rolling(window=168).mean()
    df['Max_7d'] = df['CLOSE'].rolling(window=168).max()
    df['Min_7d'] = df['CLOSE'].rolling(window=168).min()
    df['Std_7d'] = df['CLOSE'].rolling(window=168).std()
    df['Range_7d'] = df['Max_7d'] - df['Min_7d']

    df.bfill(inplace=True) 
    return df

In [148]:
def make_prediction(df, timestep=56):
    X = df.drop(columns=['DATE', 'TIME', 'datetime']).values
    X = X[-timestep:] 
    X = X.reshape(1, -1) 

    prediction = model.predict(X)
    return prediction

In [186]:
def send_prediction(chat_id, prediction):
    message = f"Предсказание: {prediction}"
    bot.send_message(chat_id, message)
    print(f"Отправлено предсказание: {prediction}")

In [238]:
def download_sber_data(chat_id):
    global df
    try:
        # Расчет дат
        end_date = datetime.today()
        start_date = end_date - timedelta(days=11)
        
        from_date = start_date.strftime("%Y%m%d")
        to_date = end_date.strftime("%Y%m%d")
        current_datetime = datetime.now()
        current_hour = current_datetime.hour
        ticker = "SBER"
        market = 1
        period = 7  # 1 час
        output_format = 5  # CSV
        file_name = f"{ticker}_{from_date}_{to_date}.csv"
        file_path = file_name  
        
        url = (
            "https://export.finam.ru/"
            f"{file_name}?market={market}&em=3&code={ticker}&apply=0&df={start_date.day}&mf={start_date.month-1}&yf={start_date.year}"
            f"&from={from_date}&dt={end_date.day}&mt={end_date.month-1}&yt={end_date.year}"
            f"&to={to_date}&p={period}&f={file_name}&e=.csv&cn={ticker}&dtf=5&tmf=4&MSOR=0&mstimever=1&sep=1&sep2=1&datf={output_format}"
        )
    
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
    
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        with open(file_path, "wb") as file:
            file.write(response.content)
    
        df = pd.read_csv(
            file_path, 
            delimiter=',',
            names=["DATE", "TIME", "OPEN", "HIGH", "LOW", "CLOSE", "VOL"], 
            header=0
        )
        df = preprocess_data(df)
        prediction = make_prediction(df)
        last_close_price = df['CLOSE'].iloc[-1]

        message = f"Цена закрытия за предыдущий час: {last_close_price} \nПредикт цены на {current_hour+1}:00 : {prediction[0]}"
        bot.send_message(chat_id, message)
        print(f"Отправлено предсказание: {prediction}")
        
        print(f"Данные загружены в DataFrame\n{message}")
    
    except Exception as e:
        print(f"Ошибка при загрузке и обработке данных: {e}")

In [251]:
def schedule_job():
    chat_id = 967457055 
    
    schedule.every().hour.at(":01").do(partial(download_sber_data(chat_id), chat_id))  
    while True:
        schedule.run_pending()
        time.sleep(1)

if __name__ == "__main__":
    thread = threading.Thread(target=schedule_job)
    thread.start()
    
    bot.polling(none_stop=True)

Exception in thread Thread-46 (schedule_job):
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\anaconda3\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ivans\AppData\Local\Temp\ipykernel_20072\452629585.py", line 4, in schedule_job
TypeError: the first argument must be callable


Отправлено предсказание: [319.16293]
Данные загружены в DataFrame
Цена закрытия за предыдущий час: 319.17 
Предикт цены на 19:00 : 319.1629333496094


ReadTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)

In [250]:
download_sber_data(967457055)

Отправлено предсказание: [318.4512]
Данные загружены в DataFrame
Цена закрытия за предыдущий час: 318.45 
Предикт цены на 19:00 : 318.4512023925781
